## Observations and Insights

#### 1) Tumor volume its mostlikely to be stay large for mice that are heavier
#### 2) Time under a treatment will not alwasys result in decresed volume of tumor; some mice experience increased volume of tumor with the time.
#### 3) Drugs Ramicane & Capomulin seem to be the most effective at reducing the volume of tumors on mice


## Dependencies and starter code

In [ ]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as st
import numpy as np
from scipy.stats import linregress

# Study data files
mouse_metadata = "data/Mouse_metadata.csv"
study_results = "data/Study_results.csv"

# Read the mouse data and the study results
mouse_metadata = pd.read_csv(mouse_metadata)
study_results = pd.read_csv(study_results)

# Combine the data into a single dataset
df= Mouse_Study_combined_df = pd.merge(study_results, mouse_metadata, on="Mouse ID")

df


## Summary statistics

In [ ]:
# Generate a summary statistics table of mean, median, variance, standard deviation, and SEM of the tumor volume for each regimen
# Generate a summary statistics table of mean, median, variance, standard deviation, and SEM of the tumor volume for each regimen
mouse_summary_df=df.groupby(['Drug Regimen'])['Tumor Volume (mm3)'].agg({'Mean':np.mean, 'SD':np.std,'Variance':np.var,'Median':np.median,'SEM':st.sem}).reset_index()
mouse_summary_df


## Bar plots

In [ ]:
# Generate a bar plot showing number of data points for each treatment regimen using pandas
drugs = mouse_summary_df['Drug Regimen']

multi_plot = mouse_summary_df.plot(kind="bar", figsize=(20,10))
multi_plot.set_xticklabels(drugs)
multi_plot.set_ylabel("Volume of Tumor in (mm3)")
multi_plot.set_xlabel("Drug")
multi_plot.set_title("Tumor Volume Statitics")
plt.show()
plt.tight_layout()

In [ ]:

# Generate a bar plot showing number of data points for each treatment regimen using pyplot
# Setting the positions and width for the bars
pos = list(range(len(mouse_summary_df['Mean'])))
width = 0.1
 
# Plotting the bars
fig, ax = plt.subplots(figsize=(20,10))
#plt.axis('off')
plt.grid(b=None)
# Create a bar with pre_score data,
# in position pos,
plt.bar(pos,
        mouse_summary_df['Mean'],
        # of width
        width,
        # with alpha 0.5
        alpha=0.5, 
        # with label the first value in first_name
        label=mouse_summary_df['Drug Regimen'][0])
# Create a bar with mid_score data,
# in position pos + some width buffer,
plt.bar([p + width for p in pos],
        #using df['mid_score'] data,
        mouse_summary_df['SD'],
        # of width
        width,
        # with alpha 0.5
        alpha=0.5,
       
        # with label the second value in first_name
        label=mouse_summary_df['Drug Regimen'][1])
# Create a bar with post_score data,
# in position pos + some width buffer,
plt.bar([p + width*2 for p in pos],
        #using df['post_score'] data,
        mouse_summary_df['Variance'],
        # of width
        width,
        # with alpha 0.5
        alpha=0.5,
        
       
        # with label the third value in first_name
        label=mouse_summary_df['Drug Regimen'][2])
plt.bar([p + width*3 for p in pos],
        #using df['post_score'] data,
        mouse_summary_df['Median'],
        # of width
        width,
        # with alpha 0.5
        alpha=0.5,
       
        # with label the third value in first_name
        label=mouse_summary_df['Drug Regimen'][3])
plt.bar([p + width*4 for p in pos],
        #using df['post_score'] data,
        mouse_summary_df['SEM'],
        # of width
        width,
        # with alpha 0.5
        alpha=0.5,
       
        # with label the third value in first_name
        label=mouse_summary_df['Drug Regimen'][4])
# Set the y axis label
ax.set_ylabel('Volume of Tumor in (mm3)')
# Set the y axis label
ax.set_xlabel('Drug')
# Set the chart's title
ax.set_title('Tumor Volume Statitics')
# Set the position of the x ticks
ax.set_xticks([p + 1.5 * width for p in pos])
# Set the labels for the x ticks
ax.set_xticklabels(mouse_summary_df['Drug Regimen'])
# Adding the legend and showing the plot
plt.legend(['Mean', 'SD', 'Variance','Median', 'SEM'], loc='upper right')
plt.grid()
plt.show()


## Pie plots

In [ ]:
# Generate a pie plot showing the distribution of female versus male mice using pandas
sex_mouse = mouse_metadata.groupby('Sex' ,as_index=False )['Mouse ID'].count().rename(columns={'Mouse ID':'Count Of Mice by Sex'})
sex_mouse.plot(kind='pie',  y = 'Count Of Mice by Sex', autopct='%1.1f%%' , startangle=140, shadow=True, labels=sex_mouse ['Sex'], legend = False, fontsize=14)

In [ ]:
# Generate a pie plot showing the distribution of female versus male mice using pyplot
# Labels for the sections of our pie chart
sex_list = sex_mouse["Sex"].values.tolist()
count_list = sex_mouse["Count Of Mice by Sex"].values.tolist()
plt.pie(count_list,  labels=sex_list, autopct="%1.1f%%", shadow=True, startangle=140)
plt.title("Count Of Mice by Sex")


## Quartiles, outliers and boxplots

In [ ]:
# Calculate the final tumor volume of each mouse across four of the most promising treatment regimens. Calculate the IQR and quantitatively determine if there are any potential outliers. 
Mouse_combined = df[['Mouse ID','Tumor Volume (mm3)']].loc[df['Drug Regimen'].isin(['Capomulin','Propriva','Ketapril','Ramicane'])]
Mouse_combined_vol = Mouse_combined.groupby('Mouse ID' ,as_index=False)['Tumor Volume (mm3)'].agg({'Last':'last'})
del Mouse_combined_vol['Mouse ID']
Mouse_combined_vol_list =  Mouse_combined_vol['Last']

quartiles =Mouse_combined_vol_list.quantile([.25,.5,.75])
lowerq = quartiles[0.25]
upperq = quartiles[0.75]
iqr = upperq-lowerq
print(f"The lower quartile of Tumor Volume (mm3) is: {lowerq}")
print(f"The upper quartile of Tumor Volume (mm3) is: {upperq}")
print(f"The interquartile range of Tumor Volume (mm3) is: {iqr}")
print(f"The  median of Tumor Volume (mm3) is: {quartiles[0.5]} ")
lower_bound = lowerq - (1.5*iqr)
upper_bound = upperq + (1.5*iqr)
print(f"Values below {lower_bound} could be outliers.")
print(f"Values above {upper_bound} could be outliers.")


In [ ]:
# Generate a box plot of the final tumor volume of each mouse across four regimens of interest
fig1, ax1 = plt.subplots()
ax1.set_title('Final tumor volume of each mouse across Drugs: Capomulin, Propriva, Ketapril, Ramicane ')
ax1.set_ylabel('Tumor Volume in mm3 (Final)) ')
ax1.boxplot(Mouse_combined_vol_list)
plt.show()

## Line and scatter plots

In [ ]:
# Generate a line plot of time point versus tumor volume for a mouse treated with Capomulin
mouse_m601 = df.loc[df['Mouse ID']=='m601']
mouse_m601.loc[mouse_m601['Drug Regimen']=='Capomulin'].plot(kind='line',x='Timepoint',y='Tumor Volume (mm3)',color='red')

In [ ]:
# Generate a scatter plot of mouse weight versus average tumor volume for the Capomulin regimen
cap_weight = df[['Weight (g)','Mouse ID']].loc[df['Drug Regimen']=='Capomulin']
x_axis = cap_weight.groupby(['Mouse ID'])['Weight (g)'].agg({'Weight':'first'})
x_axis = x_axis['Weight']
x_axis = x_axis.reset_index()
x_axis = x_axis["Weight"]
cap_vol = df[['Tumor Volume (mm3)','Mouse ID']].loc[df['Drug Regimen']=='Capomulin']
y_axis = cap_vol.groupby(['Mouse ID'])['Tumor Volume (mm3)'].agg({'Mean':np.mean})
y_axis = y_axis.reset_index()
y_axis = y_axis['Mean']
(slope, intercept, rvalue, pvalue, stderr) = linregress(x_axis, y_axis)
regress_values = x_axis * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
plt.scatter(x_axis,y_axis)
plt.plot(x_axis,regress_values,"r-")
plt.annotate(line_eq,(19,38),fontsize=15,color="red")
plt.xlabel('Mouse Weight')
plt.ylabel('Tumor Volume (mm3)')
plt.show()

In [ ]:
# Calculate the correlation coefficient and linear regression model for mouse weight and average tumor volume for the Capomulin regimen

print(f"The correlation coefficient is: {rvalue}")